In [1]:
import findspark
findspark.init()

import pyspark
from src.emulator import generate_transactions_table, add_frauds

In [2]:
spark = (
    pyspark.sql.SparkSession.builder
        .appName("notebook-update")
        .master("yarn")
        .config("spark.executor.memory", "1g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [3]:
transactions_sdf = (
    spark.read
    .format("parquet")
    .option("header", True)
    .option("separator", ",")
    .load("hdfs:///user/otus/card-fraud-detection/transactions.parquet")
)
transactions_sdf.count()

2539953

In [4]:
customer_profiles_table_sdf = (
    spark.read
    .format("parquet")
    .option("header", True)
    .option("separator", ",")
    .load("hdfs:///user/otus/card-fraud-detection/customer_profiles_table.parquet")
)
terminal_profiles_table_sdf = (
    spark.read
    .format("parquet")
    .option("header", True)
    .option("separator", ",")
    .load("hdfs:///user/otus/card-fraud-detection/terminal_profiles_table.parquet")
)
customer_profiles_table_df = customer_profiles_table_sdf.toPandas()
terminal_profiles_table_df = terminal_profiles_table_sdf.toPandas()

In [5]:
transactions_df = customer_profiles_table_df.groupby('CUSTOMER_ID').apply(lambda x : generate_transactions_table(x.iloc[0], nb_days=5)).reset_index(drop=True)
transactions_df = add_frauds(customer_profiles_table_df, terminal_profiles_table_df, transactions_df)

Number of frauds from scenario 1: 19
Number of frauds from scenario 2: 18
Number of frauds from scenario 3: 23


In [6]:
transactions_sdf = spark.createDataFrame(transactions_df) 
transactions_sdf.printSchema()

/usr/lib/spark/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- TX_DATETIME: timestamp (nullable = true)
 |-- CUSTOMER_ID: long (nullable = true)
 |-- TERMINAL_ID: long (nullable = true)
 |-- TX_AMOUNT: double (nullable = true)
 |-- TX_TIME_SECONDS: long (nullable = true)
 |-- TX_TIME_DAYS: long (nullable = true)
 |-- TX_FRAUD: long (nullable = true)
 |-- TX_FRAUD_SCENARIO: long (nullable = true)



In [7]:
transactions_sdf.write.mode("append").parquet("hdfs:///user/otus/card-fraud-detection/transactions.parquet")

In [8]:
transactions_sdf.count()

47999

In [9]:
transactions_sdf = (
    spark.read
    .format("parquet")
    .option("header", True)
    .option("separator", ",")
    .load("hdfs:///user/otus/card-fraud-detection/transactions.parquet")
)
transactions_sdf.count()

2587952

In [10]:
spark.stop()